# Necessary libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline, InterpolatedUnivariateSpline, Akima1DInterpolator, interp1d
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
#from datetime import timedelta
#import datetime
#import fbprophet
#from matplotlib import pyplot
from pandas import to_datetime
from statsmodels.tsa.arima_model import ARIMA
#from fbprophet import Prophet
#from pmdarima.arima import auto_arima
#import pmdarima
from scipy import interpolate
#from fbprophet import Prophet
from matplotlib import pyplot
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import plotly.graph_objs as go
import sklearn.model_selection
from scipy import stats
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline as CS
from scipy.interpolate import RBFInterpolator
from pandas._libs.tslibs.parsing import guess_datetime_format
from scipy.interpolate import Rbf
from scipy.interpolate import KroghInterpolator
import random
import math

2023-05-23 10:46:56.325811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-23 10:46:56.325830: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [70]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [71]:
df_for=pd.read_csv("5_df_for_interpolation_may11.csv")
df_for=df_for.drop(["Unnamed: 0","level_0"],axis=1)
df_for

,Date,Area,Wound,Gender,Race,Ethinicity,Age
0,2016/10/26,4.0000,1,M,White,Non-Hispanic,71.969167
1,2016/11/01,4.0000,1,M,White,Non-Hispanic,71.969167
2,2016/11/02,4.0000,1,M,White,Non-Hispanic,71.969167
3,2016/11/07,7.5000,1,M,White,Non-Hispanic,71.969167
4,2016/11/09,4.0000,1,M,White,Non-Hispanic,71.969167
...,...,...,...,...,...,...,...
159041,2023/01/03,2.7064,17884,F,White,Non-Hispanic,71.230000
159042,2023/01/09,2.7064,17884,F,White,Non-Hispanic,71.230000
159043,2023/01/10,8.9401,17885,M,White,Non-Hispanic,68.370000
159044,2023/01/10,5.2528,17886,M,White,Non-Hispanic,68.370000


## Defining the necessary functions

In [72]:
def get_date_and_area(df_patient_1_wound):
    df_patient_1_wound['Date'] = pd.to_datetime(df_patient_1_wound.Date,utc=True)
    df_patient_1_wound['Date'] = df_patient_1_wound['Date'].dt.strftime('%Y/%m/%d')
    #display(df_patient_1_wound)
    area=df_patient_1_wound["Area"].values
    area=area.tolist()
    date=df_patient_1_wound["Date"].values
    date=date.tolist()
    return (area,date)

In [73]:
def indexing_dataframe(date,area):
    df=pd.DataFrame()
    df["Date"]=date
    df["Area"]=area
    df=df.drop_duplicates(subset=['Date'], keep='first')
    df=df.reset_index()
    df=df.drop(["index"],axis=1)
    

    dff=df.set_index('Date')
    #print("dff shape",dff.shape[0])
    return (dff,df)

In [74]:
def interpolating_dates(dff,df):
    idx = pd.date_range(df["Date"][0], df["Date"][df.shape[0]-1])
    #print(idx)
    #print(len(idx))
    
    #if len(idx)>1000:
        
    s=dff.squeeze()
    s.index = pd.DatetimeIndex(s.index)
    s = s.reindex(idx, fill_value=np.nan)
    
    return (s)

## Interpolating with Polyfit

In [75]:
def interpolating_areas_polyfit(s):
    df=pd.DataFrame(s)
    df=df.reset_index()

    x=df.index.values.tolist()
    y=df.Area.values.tolist()
    #display(df["index"])

    df1=df.dropna()

    xx=df1.index.values.tolist()
    yy=df1.Area.values.tolist()

    
#     print(xx)
#     print(yy)
#     if yy>1000:
#         yy
    #cs = CS(xx, yy)
    poly=np.poly1d(np.polyfit(xx,yy,3))
    print(poly)
    #rbf = Rbf(xx, yy,kernel='multiquadric')

#     plt.plot(xx, yy, label='scipy')
#     plt.plot(x, cs(x), label='scipy')
#     plt.show()
    
    df_interpolated=pd.DataFrame({'Date':df["index"],'Area':poly(x)})
    #print("Number of visit days after interpolation is:",df_interpolated.shape[0])
    x=df_interpolated["Area"].values
    #print(x)
    logLik = sum( np.log( prob( x, np.mean( x ), np.std( x ) ) ) ) 
    K = 1
    AIC=2*K - 2*( logLik )
    
    return df_interpolated, AIC

## Interpolating with Akima

In [76]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

# regr = OLS(y, add_constant(X)).fit()
# print(regr.aic)

import scipy.stats

def prob(y, mean, sd):
    return scipy.stats.norm( mean, sd ).pdf( y )

def interpolating_areas_akima(s):
    df=pd.DataFrame(s)
    df=df.reset_index()

    x=df.index.values.tolist()
    y=df.Area.values.tolist()
    #display(df["index"])

    df1=df.dropna()

    xx=df1.index.values.tolist()
    yy=df1.Area.values.tolist()

    
#     print(xx)
#     print(yy)
#     if yy>1000:
#         yy
    #cs = CS(xx, yy)
    #display(xx)
    #display(yy)
    akima=Akima1DInterpolator(xx,yy)
    #rbf = Rbf(xx, yy,kernel='multiquadric')

#     plt.plot(xx, yy, label='scipy')
#     plt.plot(x, cs(x), label='scipy')
#     plt.show()
    
    df_interpolated=pd.DataFrame({'Date':df["index"],'Area':akima(x)})
    print("Number of visit days after interpolation is:",df_interpolated.shape[0])
    #display(df_interpolated)
    x=df_interpolated["Area"].values
    #print(x)
    logLik = sum( np.log(prob( x, np.mean( x ), np.std( x ) ) ) )
    K = 1
    AIC=2*K - 2*( logLik )
    
    return df_interpolated, AIC

## interpolating with Krogh

In [77]:
def interpolating_areas_krogh(s):
    df=pd.DataFrame(s)
    df=df.reset_index()

    x=df.index.values.tolist()
    y=df.Area.values.tolist()
    #display(df["index"])

    df1=df.dropna()

    xx=df1.index.values.tolist()
    yy=df1.Area.values.tolist()

    
#     print(xx)
#     print(yy)
#     if yy>1000:
#         yy
    #cs = CS(xx, yy)
    krogh=KroghInterpolator(xx,yy)
    #rbf = Rbf(xx, yy,kernel='multiquadric')

#     plt.plot(xx, yy, label='scipy')
#     plt.plot(x, cs(x), label='scipy')
#     plt.show()
    
    df_interpolated=pd.DataFrame({'Date':df["index"],'Area':krogh(x)})
    #print("Number of visit days after interpolation is:",df_interpolated.shape[0])
    x=df_interpolated["Area"].values
    #print(x)
    logLik = sum( np.log( prob( x, np.mean( x ), np.std( x ) ) ) ) 
    K = 1
    AIC=2*K - 2*( logLik )
    
    return df_interpolated, AIC

## Interpolating with CubicSpline

In [78]:
def interpolating_areas_cubic(s):
    df=pd.DataFrame(s)
    df=df.reset_index()

    x=df.index.values.tolist()
    y=df.Area.values.tolist()
    #display(df["index"])

    df1=df.dropna()

    xx=df1.index.values.tolist()
    yy=df1.Area.values.tolist()

    
#     print(xx)
#     print(yy)
#     if yy>1000:
#         yy
    cs = CS(xx, yy)
    #krogh=KroghInterpolator(xx,yy)
    #rbf = Rbf(xx, yy,kernel='multiquadric')

#     plt.plot(xx, yy, label='scipy')
#     plt.plot(x, cs(x), label='scipy')
#     plt.show()
    
    df_interpolated=pd.DataFrame({'Date':df["index"],'Area':cs(x)})
    #print("Number of visit days after interpolation is:",df_interpolated.shape[0])
    x=df_interpolated["Area"].values
    #print(x)
    logLik = sum( np.log( prob( x, np.mean( x ), np.std( x ) ) ) ) 
    K = 1
    AIC=2*K - 2*( logLik )
    
    return df_interpolated, AIC

## Linear Interpolation

In [79]:
def interpolating_areas_linear(s):
    df=pd.DataFrame(s)
    df=df.reset_index()

    x=df.index.values.tolist()
    y=df.Area.values.tolist()
    #display(df["index"])

    df1=df.dropna()

    xx=df1.index.values.tolist()
    yy=df1.Area.values.tolist()

    
#     print(xx)
#     print(yy)
#     if yy>1000:
#         yy
    #cs = CS(xx, yy)
    lin=interp1d(xx,yy)
    #rbf = Rbf(xx, yy,kernel='multiquadric')

#     plt.plot(xx, yy, label='scipy')
#     plt.plot(x, cs(x), label='scipy')
#     plt.show()
    
    df_interpolated=pd.DataFrame({'Date':df["index"],'Area':lin(x)})
    #print("Number of visit days after interpolation is:",df_interpolated.shape[0])

    x=df_interpolated["Area"].values
    #print(x)
    logLik = sum( np.log( prob( x, np.mean( x ), np.std( x ) ) ) ) 
    K = 1
    AIC=2*K - 2*( logLik )
    
    return df_interpolated, AIC

## Interpolating with RBF

In [80]:
def interpolating_areas_rbf(s):
    df=pd.DataFrame(s)
    df=df.reset_index()

    x=df.index.values.tolist()
    y=df.Area.values.tolist()
    #display(df["index"])

    df1=df.dropna()

    xx=df1.index.values.tolist()
    yy=df1.Area.values.tolist()

    
#     print(xx)
#     print(yy)
#     if yy>1000:
#         yy
    #cs = CS(xx, yy)
    #krogh=KroghInterpolator(xx,yy)
    rbf = Rbf(xx, yy,kernel='multiquadric')

#     plt.plot(xx, yy, label='scipy')
#     plt.plot(x, cs(x), label='scipy')
#     plt.show()
    
    df_interpolated=pd.DataFrame({'Date':df["index"],'Area':rbf(x)})
    #print("Number of visit days after interpolation is:",df_interpolated.shape[0])

    x=df_interpolated["Area"].values
    #print(x)
    logLik = sum( np.log( prob( x, np.mean( x ), np.std( x ) ) ) ) 
    K = 1
    AIC=2*K - 2*( logLik )
    
    return df_interpolated, AIC

In [81]:
# list_wounds=df_for["Wound"].unique().tolist()
# wound=0
# df_merged=pd.DataFrame()

# for i in range(len(list_wounds)):
# #     if i>4:
# #         break
#     wound_number=list_wounds[i]
#     df_indivi_patient=df_for.query("Wound==@wound_number")
#     print("Wound",i+1)
#     gender=df_indivi_patient["Gender"].unique()
#     race=df_indivi_patient["Race"].unique()
#     ethnicity=df_indivi_patient["Ethinicity"].unique()


#     df_patient_1_wound=df_indivi_patient

#     df_patient_1_wound=df_patient_1_wound.sort_values(by=['Date'])
#     df_patient_1_wound=df_patient_1_wound.reset_index()
#     df_patient_1_wound=df_patient_1_wound.drop(["index"],axis=1)

#     df_patient_1_wound=df_patient_1_wound.dropna()
#     df_patient_1_wound=df_patient_1_wound.reset_index()
#     df_patient_1_wound=df_patient_1_wound.drop(["index"],axis=1)

#     if df_patient_1_wound.shape[0]>1:
#         #Do the interpolation for all dates
#         area, date= get_date_and_area(df_patient_1_wound)    
#         dff,df=indexing_dataframe(date,area)
#         if df.shape[0]>1:
#             print("Number of visit days before interpolation is:",df.shape[0])
#             df=df.sort_values(by=['Date'])
#             dff=dff.sort_values(by=['Date'])

#             s=interpolating_dates(dff,df)
            
#             #moving towards with the first interpolation
            
#             df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
#             df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
#             df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
#             df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            
            
            
#             print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)
            
#             AIC_list=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]
            
#             if min(AIC_list)==AIC_akima:
#                 print("For this wound, choosing AKIMA as the best interpolator", "\n")
#                 df_interpolated=df_interpolated_akima
#             elif min(AIC_list)==AIC_rbf:
#                 print("For this wound, choosing RBF as the best interpolator", "\n")
#                 df_interpolated=df_interpolated_rbf
#             elif min(AIC_list)==AIC_krogh:
#                 print("For this wound, choosing Krogh as the best interpolator", "\n")
#                 df_interpolated=df_interpolated_krogh
#             elif min(AIC_list)==AIC_linear:
#                 print("For this wound, choosing Linear as the best interpolator", "\n")
#                 df_interpolated=df_interpolated_linear
# #             df_interpolated=interpolating_areas_akima(s)
# #             if max(df_interpolated["Area"])>100:
# #                 df_interpolated=interpolating_areas_linear(s)



#             df=s.to_frame().reset_index()
#             wound=wound+1
#             df_interpolated["Wound"]=wound
#             df_interpolated["Gender"]=gender[0]
#             df_interpolated["Race"]=race[0]
#             df_interpolated["Ethinicity"]=ethnicity[0]

#             df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

#             #Plotting the interpolation

#             x=df.index.values
#             y=df.Area.values

#             xnew=df_interpolated.index.values
#             ynew=df_interpolated.Area.values
#             plt.plot(x, y, 'o', xnew, ynew, '-')
#             plt.plot(x,y,'o')
#             plt.show()
# #                 if df_interpolated.shape[0]==587:
# #                     break
# #     if df_interpolated.shape[0]==587:
# #         break

#             #display(df_interpolated)


In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
#     if (i+1)==3910:
    wound_number=list_wounds[i]
    df_indivi_patient=df_for.query("Wound==@wound_number")
    print(color.BOLD+"\n","Wound",str(i+1)+color.END)
    number=i+1

    gender=df_indivi_patient["Gender"].unique()
    race=df_indivi_patient["Race"].unique()
    ethnicity=df_indivi_patient["Ethinicity"].unique()


    df_patient_1_wound_part_1=df_indivi_patient

    df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
    df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
    df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

    df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
    df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
    df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




    #Do the interpolation for all dates
    area, date= get_date_and_area(df_patient_1_wound_part_1)    
    #print(area,date)
    dff,df=indexing_dataframe(date,area)
    if df.shape[0]>1:
        print("Number of visit days before interpolation is:",df.shape[0])
        df=df.sort_values(by=['Date'])

        dff=dff.sort_values(by=['Date'])

        s=interpolating_dates(dff,df)

        df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
        df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
        df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
        df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

        print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

        AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
        AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

        if len(AIC_list_part_1)==0:
            print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
            df_interpolated=df_interpolated_linear
            interpolation_method.append('Linear')
        else:
            if min(AIC_list_part_1)==AIC_akima:
                print("For this wound, choosing AKIMA as the best interpolator", "\n")
                df_interpolated=df_interpolated_akima
                interpolation_method.append('AKIMA')
            elif min(AIC_list_part_1)==AIC_rbf:
                print("For this wound, choosing RBF as the best interpolator", "\n")
                df_interpolated=df_interpolated_rbf
                interpolation_method.append('RBF')

            elif min(AIC_list_part_1)==AIC_krogh:
                print("For this wound, choosing Krogh as the best interpolator", "\n")
                df_interpolated=df_interpolated_krogh
                interpolation_method.append('Krogh')
            elif min(AIC_list_part_1)==AIC_linear:
                print("For this wound, choosing Linear as the best interpolator", "\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')



        df=s.to_frame().reset_index()
        df_interpolated["Gender"]=gender[0]
        df_interpolated["Race"]=race[0]
        df_interpolated["Ethinicity"]=ethnicity[0]

        df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

        #Plotting the interpolation

        x=df.index.values
        y=df.Area.values

        xnew=df_interpolated.index.values
        ynew=df_interpolated.Area.values
        plt.plot(x, y, 'o', xnew, ynew, '-')
        #plt.plot(x,y,'o')
        plt.show()


        dfA=df_interpolated

        start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
        print("Start",start)
        end=start-1+math.ceil(len(dfA.index.values)*0.2)
        print("End",end)
        dfA_1=dfA.drop(dfA.index[start:end])

        data_wound=dfA_1
        df_indivi_patient=data_wound
        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()

        df_patient_1_wound_part_2=df_indivi_patient

        df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
        df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
        df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

        df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
        df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
        df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_2)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)

        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

            if len(AIC_list_part_2)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
            else:
                if min(AIC_list_part_2)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    AIC_final=AIC_akima
                elif min(AIC_list_part_2)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    AIC_final=AIC_rbf
                elif min(AIC_list_part_2)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    AIC_final=AIC_krogh
                elif min(AIC_list_part_2)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    AIC_final=AIC_linear

            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            plt.plot(x,y,'o')
            plt.show()

            dfB_1=df_interpolated

            first=dfA["Area"].values
            second=dfB_1["Area"].values

            display("Test values from the first interpolation (now considered as actual)",first[start:end])
            display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

            first_test_values=first[start:end]
            second_test_values=second[start:end]

            mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
            rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
            corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
            r_square= corr**2

            print("MAPE:",round(mape,4))
            print("RMSE",round(rmse,4))
            print("R-squarred",round(r_square,4))
            print("AIC",round(AIC_final,4))

            all_mape.append(mape)
            all_rmse.append(rmse)
            all_rsquarred.append(r_square)
            all_aic.append(AIC_final)
            all_wound.append(number)

In [ ]:
interpolation_method

In [84]:
df_merged.to_csv("6_df_interpolated_for_forecasting_may22.csv")

In [85]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric=pd.DataFrame(lists).transpose()
df_metric.columns=["RMSE","MAPE","AIC","Wound Number","R-squarred",'Method']
#df_metric=pd.DataFrame(all_mape, columns=["MAPE"])
# df_metric=df_metric.replace({np.inf: np.nan})
# df_metric=df_metric.dropna()
# df_metric

# display(df_metric.sort_values('MAPE').tail(50))

# display(df_metric.describe())
df_metric

,RMSE,MAPE,AIC,Wound Number,R-squarred,Method
0,1.156098,0.638773,576.691058,1,0.639338,Linear
1,0.0,0.0,278.083492,2,1.0,Linear
2,0.006555,0.007433,-43.18967,3,1.0,RBF
3,0.024748,0.017861,53.521066,5,0.999942,Linear
4,0.026651,0.068398,11.67976,6,0.997253,Krogh
...,...,...,...,...,...,...
14913,0.0,0.0,36.477264,17877,1.0,Linear
14914,0.004448,0.011955,-38.374363,17879,NaN,RBF
14915,0.0,0.0,57.542496,17880,1.0,Linear
14916,8.349745,0.053134,60.431775,17881,NaN,RBF


In [86]:
df_metric.to_csv("df_metric_may23.csv")

In [91]:
df_metric=pd.read_csv("df_metric_may23.csv")
df_metric=df_metric.rename(columns={'R-squarred':'R'})
df_metric=df_metric.drop(["Unnamed: 0"],axis=1)
display("Raw",df_metric.describe())

count=0
for i in range(df_metric.shape[0]):
    if df_metric["RMSE"][i]==0 and str(df_metric["MAPE"][i])=='nan':
        df_metric["MAPE"][i]=0
        count=count+1
display(df_metric.describe())
print(count)

count=0
for i in range(df_metric.shape[0]):
    if df_metric["RMSE"][i]==0:
        if df_metric["MAPE"][i]==0:
            if str(df_metric["R"][i])=="nan":
                df_metric["R"][i]=1
                count=count+1
display(df_metric.describe())
print(count)

df_metric=df_metric.dropna(subset=['RMSE','MAPE'])
display("Removing all NaN",df_metric.describe())


# #df_metric_1=df_metric.dropna()
df_metric=df_metric.reset_index()
df_metric=df_metric.drop(["index"],axis=1)
df_metric
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric[(df_metric['RMSE'] >= 5) | (df_metric['MAPE'] >= 5)]
high_mape_df

'Raw'

,RMSE,MAPE,AIC,Wound Number,R
count,14572.000000,1.431400e+04,14918.000000,14918.000000,12685.000000
mean,2.035628,inf,321.368116,9284.130446,0.803633
std,15.653559,NaN,1056.422529,5100.631554,0.286673
min,0.000000,0.000000e+00,-15472.884087,1.000000,0.000000
25%,0.001479,3.062525e-03,-5.653944,4952.250000,0.716507
50%,0.056500,5.201960e-02,95.356828,9516.500000,0.958014
75%,0.490996,2.281883e-01,342.200029,13669.750000,1.000000
max,1029.100620,inf,22836.782518,17884.000000,1.000000


,RMSE,MAPE,AIC,Wound Number,R
count,14572.000000,1.457100e+04,14918.000000,14918.000000,12685.000000
mean,2.035628,inf,321.368116,9284.130446,0.803633
std,15.653559,NaN,1056.422529,5100.631554,0.286673
min,0.000000,0.000000e+00,-15472.884087,1.000000,0.000000
25%,0.001479,2.056057e-03,-5.653944,4952.250000,0.716507
50%,0.056500,4.835733e-02,95.356828,9516.500000,0.958014
75%,0.490996,2.214133e-01,342.200029,13669.750000,1.000000
max,1029.100620,inf,22836.782518,17884.000000,1.000000


257


,RMSE,MAPE,AIC,Wound Number,R
count,14572.000000,1.457100e+04,14918.000000,14918.000000,13446.000000
mean,2.035628,inf,321.368116,9284.130446,0.814747
std,15.653559,NaN,1056.422529,5100.631554,0.282115
min,0.000000,0.000000e+00,-15472.884087,1.000000,0.000000
25%,0.001479,2.056057e-03,-5.653944,4952.250000,0.740411
50%,0.056500,4.835733e-02,95.356828,9516.500000,0.972384
75%,0.490996,2.214133e-01,342.200029,13669.750000,1.000000
max,1029.100620,inf,22836.782518,17884.000000,1.000000


761


'Removing all NaN'

,RMSE,MAPE,AIC,Wound Number,R
count,14571.000000,1.457100e+04,14571.000000,14571.000000,13446.000000
mean,2.035759,inf,327.533321,9280.677167,0.814747
std,15.654088,NaN,1066.986719,5109.407577,0.282115
min,0.000000,0.000000e+00,-15472.884087,1.000000,0.000000
25%,0.001479,2.056057e-03,-3.795089,4934.000000,0.740411
50%,0.056472,4.835733e-02,99.444126,9517.000000,0.972384
75%,0.491092,2.214133e-01,348.653994,13673.000000,1.000000
max,1029.100620,inf,22836.782518,17884.000000,1.000000


,RMSE,MAPE,AIC,Wound Number,R,Method
9,0.308281,inf,440.331525,12,0.899594,Linear
27,0.000813,inf,-696.653827,33,NaN,Linear
33,48.766337,0.318813,4973.305697,42,0.011343,Linear
36,0.047635,inf,645.313564,46,0.322717,Linear
40,0.864356,13.828814,85.627356,52,1.000000,Linear
...,...,...,...,...,...,...
14380,26.165381,0.432178,729.327603,17681,0.653577,Linear
14386,5.466560,0.037108,1016.348449,17687,0.843686,Linear
14428,11.798471,0.268699,754.579718,17730,0.803961,Linear
14472,13.206456,0.092829,497.999714,17777,0.952778,Linear


In [92]:
list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

## Now we will be dropping the rows from the metric dataframe because we will be re-running the interpolation and adding new metric in the dataframe for certain wounds

In [93]:
df_metric_1 = df_metric.drop(df_metric.index[list_index])
df_metric_1.describe()

,RMSE,MAPE,AIC,Wound Number,R
count,13070.000000,13070.000000,13070.000000,13070.000000,12067.000000
mean,0.387570,0.188467,230.980049,9212.462204,0.837909
std,0.820235,0.488333,815.761630,5138.929478,0.267321
min,0.000000,0.000000,-15472.884087,1.000000,0.000000
25%,0.000455,0.000629,-9.114372,4792.250000,0.793381
50%,0.039868,0.031984,83.961281,9383.000000,0.986101
75%,0.316029,0.151680,287.574217,13665.750000,1.000000
max,4.999258,4.997134,16160.661249,17884.000000,1.000000


## 2nd run of interpolation on 1501 wounds

In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
    if (i+1) in high_mapes:
        wound_number=list_wounds[i]
        df_indivi_patient=df_for.query("Wound==@wound_number")
        print(color.BOLD+"\n","Wound",str(i+1)+color.END)
        number=i+1

        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()


        df_patient_1_wound_part_1=df_indivi_patient

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_1)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)
        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

            if len(AIC_list_part_1)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')
            else:
                if min(AIC_list_part_1)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    interpolation_method.append('AKIMA')
                elif min(AIC_list_part_1)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    interpolation_method.append('RBF')

                elif min(AIC_list_part_1)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    interpolation_method.append('Krogh')
                elif min(AIC_list_part_1)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    interpolation_method.append('Linear')



            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            #plt.plot(x,y,'o')
            plt.show()


            dfA=df_interpolated

            start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
            print("Start",start)
            end=start-1+math.ceil(len(dfA.index.values)*0.2)
            print("End",end)
            dfA_1=dfA.drop(dfA.index[start:end])

            data_wound=dfA_1
            df_indivi_patient=data_wound
            gender=df_indivi_patient["Gender"].unique()
            race=df_indivi_patient["Race"].unique()
            ethnicity=df_indivi_patient["Ethinicity"].unique()

            df_patient_1_wound_part_2=df_indivi_patient

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound_part_2)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)

            if df.shape[0]>1:
                print("Number of visit days before interpolation is:",df.shape[0])
                df=df.sort_values(by=['Date'])

                dff=dff.sort_values(by=['Date'])

                s=interpolating_dates(dff,df)

                df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
                df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
                df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
                df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

                print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

                AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
                AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

                if len(AIC_list_part_2)==0:
                    print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                    df_interpolated=df_interpolated_linear
                else:
                    if min(AIC_list_part_2)==AIC_akima:
                        print("For this wound, choosing AKIMA as the best interpolator", "\n")
                        df_interpolated=df_interpolated_akima
                        AIC_final=AIC_akima
                    elif min(AIC_list_part_2)==AIC_rbf:
                        print("For this wound, choosing RBF as the best interpolator", "\n")
                        df_interpolated=df_interpolated_rbf
                        AIC_final=AIC_rbf
                    elif min(AIC_list_part_2)==AIC_krogh:
                        print("For this wound, choosing Krogh as the best interpolator", "\n")
                        df_interpolated=df_interpolated_krogh
                        AIC_final=AIC_krogh
                    elif min(AIC_list_part_2)==AIC_linear:
                        print("For this wound, choosing Linear as the best interpolator", "\n")
                        df_interpolated=df_interpolated_linear
                        AIC_final=AIC_linear

                df=s.to_frame().reset_index()
                df_interpolated["Gender"]=gender[0]
                df_interpolated["Race"]=race[0]
                df_interpolated["Ethinicity"]=ethnicity[0]

                #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

                #Plotting the interpolation

                x=df.index.values
                y=df.Area.values

                xnew=df_interpolated.index.values
                ynew=df_interpolated.Area.values
                plt.plot(x, y, 'o', xnew, ynew, '-')
                plt.plot(x,y,'o')
                plt.show()

                dfB_1=df_interpolated

                first=dfA["Area"].values
                second=dfB_1["Area"].values

                display("Test values from the first interpolation (now considered as actual)",first[start:end])
                display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

                first_test_values=first[start:end]
                second_test_values=second[start:end]

                mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
                rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
                corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
                r_square= corr**2

                print("MAPE:",round(mape,4))
                print("RMSE",round(rmse,4))
                print("R-squarred",round(r_square,4))
                print("AIC",round(AIC_final,4))

                all_mape.append(mape)
                all_rmse.append(rmse)
                all_rsquarred.append(r_square)
                all_aic.append(AIC_final)
                all_wound.append(number)

In [95]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric_new1=pd.DataFrame(lists).transpose()
df_metric_new1.columns=["RMSE","MAPE","AIC","Wound Number","R","Method"]
df_metric_new1

count=0
for i in range(df_metric_new1.shape[0]):
    if df_metric_new1["RMSE"][i]==0 and str(df_metric_new1["MAPE"][i])=='nan':
        df_metric_new1["MAPE"][i]=0
        count=count+1
display(df_metric_new1.describe())
#print(count)

count=0
for i in range(df_metric_new1.shape[0]):
    if df_metric_new1["RMSE"][i]==0:
        if df_metric_new1["MAPE"][i]==0:
            if str(df_metric_new1["R"][i])=="nan":
                df_metric_new1["R"][i]=1
                count=count+1
#display(df_metric.describe())
#print(count)


# #df_metric_1=df_metric.dropna()
df_metric_new1=df_metric_new1.reset_index()
df_metric_new1=df_metric_new1.drop(["index"],axis=1)
df_metric_new1
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric_new1[(df_metric_new1['RMSE'] >= 5) | (df_metric_new1['MAPE'] >= 5)]
high_mape_df

list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

df_metric_2 = df_metric_new1.drop(df_metric_new1.index[list_index])
df_metric_2.describe()

,RMSE,MAPE,AIC,Wound Number,R,Method
count,1501.0,1499.0,1501.000000,1501,1355.0,1501
unique,1395.0,1210.0,1499.000000,1501,1143.0,4
top,0.0,inf,112.032203,12,1.0,Linear
freq,103.0,188.0,2.000000,1,151.0,1152


,RMSE,MAPE,AIC,Wound Number,R,Method
count,813.0,812.0,813.000000,813,780.0,813
unique,707.0,710.0,812.000000,813,547.0,4
top,0.0,0.0,112.032203,12,1.0,Linear
freq,103.0,103.0,2.000000,1,189.0,608


## 3rd run of interpolation on 688 wounds

In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
    if (i+1) in high_mapes:
        wound_number=list_wounds[i]
        df_indivi_patient=df_for.query("Wound==@wound_number")
        print(color.BOLD+"\n","Wound",str(i+1)+color.END)
        number=i+1

        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()


        df_patient_1_wound_part_1=df_indivi_patient

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_1)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)
        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

            if len(AIC_list_part_1)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')
            else:
                if min(AIC_list_part_1)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    interpolation_method.append('AKIMA')
                elif min(AIC_list_part_1)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    interpolation_method.append('RBF')

                elif min(AIC_list_part_1)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    interpolation_method.append('Krogh')
                elif min(AIC_list_part_1)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    interpolation_method.append('Linear')



            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            #plt.plot(x,y,'o')
            plt.show()


            dfA=df_interpolated

            start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
            print("Start",start)
            end=start-1+math.ceil(len(dfA.index.values)*0.2)
            print("End",end)
            dfA_1=dfA.drop(dfA.index[start:end])

            data_wound=dfA_1
            df_indivi_patient=data_wound
            gender=df_indivi_patient["Gender"].unique()
            race=df_indivi_patient["Race"].unique()
            ethnicity=df_indivi_patient["Ethinicity"].unique()

            df_patient_1_wound_part_2=df_indivi_patient

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound_part_2)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)

            if df.shape[0]>1:
                print("Number of visit days before interpolation is:",df.shape[0])
                df=df.sort_values(by=['Date'])

                dff=dff.sort_values(by=['Date'])

                s=interpolating_dates(dff,df)

                df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
                df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
                df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
                df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

                print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

                AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
                AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

                if len(AIC_list_part_2)==0:
                    print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                    df_interpolated=df_interpolated_linear
                else:
                    if min(AIC_list_part_2)==AIC_akima:
                        print("For this wound, choosing AKIMA as the best interpolator", "\n")
                        df_interpolated=df_interpolated_akima
                        AIC_final=AIC_akima
                    elif min(AIC_list_part_2)==AIC_rbf:
                        print("For this wound, choosing RBF as the best interpolator", "\n")
                        df_interpolated=df_interpolated_rbf
                        AIC_final=AIC_rbf
                    elif min(AIC_list_part_2)==AIC_krogh:
                        print("For this wound, choosing Krogh as the best interpolator", "\n")
                        df_interpolated=df_interpolated_krogh
                        AIC_final=AIC_krogh
                    elif min(AIC_list_part_2)==AIC_linear:
                        print("For this wound, choosing Linear as the best interpolator", "\n")
                        df_interpolated=df_interpolated_linear
                        AIC_final=AIC_linear

                df=s.to_frame().reset_index()
                df_interpolated["Gender"]=gender[0]
                df_interpolated["Race"]=race[0]
                df_interpolated["Ethinicity"]=ethnicity[0]

                #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

                #Plotting the interpolation

                x=df.index.values
                y=df.Area.values

                xnew=df_interpolated.index.values
                ynew=df_interpolated.Area.values
                plt.plot(x, y, 'o', xnew, ynew, '-')
                plt.plot(x,y,'o')
                plt.show()

                dfB_1=df_interpolated

                first=dfA["Area"].values
                second=dfB_1["Area"].values

                display("Test values from the first interpolation (now considered as actual)",first[start:end])
                display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

                first_test_values=first[start:end]
                second_test_values=second[start:end]

                mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
                rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
                corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
                r_square= corr**2

                print("MAPE:",round(mape,4))
                print("RMSE",round(rmse,4))
                print("R-squarred",round(r_square,4))
                print("AIC",round(AIC_final,4))

                all_mape.append(mape)
                all_rmse.append(rmse)
                all_rsquarred.append(r_square)
                all_aic.append(AIC_final)
                all_wound.append(number)

In [97]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric_new2=pd.DataFrame(lists).transpose()
df_metric_new2.columns=["RMSE","MAPE","AIC","Wound Number","R","Method"]
df_metric_new2

count=0
for i in range(df_metric_new2.shape[0]):
    if df_metric_new2["RMSE"][i]==0 and str(df_metric_new2["MAPE"][i])=='nan':
        df_metric_new2["MAPE"][i]=0
        count=count+1
display(df_metric_new2.describe())
print(count)

count=0
for i in range(df_metric_new2.shape[0]):
    if df_metric_new2["RMSE"][i]==0:
        if df_metric_new2["MAPE"][i]==0:
            if str(df_metric_new2["R"][i])=="nan":
                df_metric_new2["R"][i]=1
                count=count+1
#display(df_metric.describe())
print(count)


# #df_metric_1=df_metric.dropna()
df_metric_new2=df_metric_new2.reset_index()
df_metric_new2=df_metric_new2.drop(["index"],axis=1)
df_metric_new2
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric_new2[(df_metric_new2['RMSE'] >= 5) | (df_metric_new2['MAPE'] >= 5)]
high_mape_df

list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

df_metric_3 = df_metric_new2.drop(df_metric_new2.index[list_index])
df_metric_3.describe()

,RMSE,MAPE,AIC,Wound Number,R,Method
count,688.0,687.0,688.000000,688,623.0,688
unique,648.0,557.0,688.000000,688,562.0,4
top,0.0,inf,5028.007284,42,1.0,Linear
freq,41.0,90.0,1.000000,1,50.0,544


24
33


,RMSE,MAPE,AIC,Wound Number,R,Method
count,254.0,253.0,254.00000,254,245.0,254
unique,214.0,212.0,254.00000,254,168.0,4
top,0.0,0.0,2471.25532,182,1.0,Linear
freq,41.0,41.0,1.00000,1,69.0,202


## 4th run of interpolation on 434 wounds

In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
    if (i+1) in high_mapes:
        wound_number=list_wounds[i]
        df_indivi_patient=df_for.query("Wound==@wound_number")
        print(color.BOLD+"\n","Wound",str(i+1)+color.END)
        number=i+1

        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()


        df_patient_1_wound_part_1=df_indivi_patient

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_1)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)
        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

            if len(AIC_list_part_1)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')
            else:
                if min(AIC_list_part_1)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    interpolation_method.append('AKIMA')
                elif min(AIC_list_part_1)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    interpolation_method.append('RBF')

                elif min(AIC_list_part_1)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    interpolation_method.append('Krogh')
                elif min(AIC_list_part_1)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    interpolation_method.append('Linear')



            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            #plt.plot(x,y,'o')
            plt.show()


            dfA=df_interpolated

            start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
            print("Start",start)
            end=start-1+math.ceil(len(dfA.index.values)*0.2)
            print("End",end)
            dfA_1=dfA.drop(dfA.index[start:end])

            data_wound=dfA_1
            df_indivi_patient=data_wound
            gender=df_indivi_patient["Gender"].unique()
            race=df_indivi_patient["Race"].unique()
            ethnicity=df_indivi_patient["Ethinicity"].unique()

            df_patient_1_wound_part_2=df_indivi_patient

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound_part_2)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)

            if df.shape[0]>1:
                print("Number of visit days before interpolation is:",df.shape[0])
                df=df.sort_values(by=['Date'])

                dff=dff.sort_values(by=['Date'])

                s=interpolating_dates(dff,df)

                df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
                df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
                df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
                df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

                print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

                AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
                AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

                if len(AIC_list_part_2)==0:
                    print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                    df_interpolated=df_interpolated_linear
                else:
                    if min(AIC_list_part_2)==AIC_akima:
                        print("For this wound, choosing AKIMA as the best interpolator", "\n")
                        df_interpolated=df_interpolated_akima
                        AIC_final=AIC_akima
                    elif min(AIC_list_part_2)==AIC_rbf:
                        print("For this wound, choosing RBF as the best interpolator", "\n")
                        df_interpolated=df_interpolated_rbf
                        AIC_final=AIC_rbf
                    elif min(AIC_list_part_2)==AIC_krogh:
                        print("For this wound, choosing Krogh as the best interpolator", "\n")
                        df_interpolated=df_interpolated_krogh
                        AIC_final=AIC_krogh
                    elif min(AIC_list_part_2)==AIC_linear:
                        print("For this wound, choosing Linear as the best interpolator", "\n")
                        df_interpolated=df_interpolated_linear
                        AIC_final=AIC_linear

                df=s.to_frame().reset_index()
                df_interpolated["Gender"]=gender[0]
                df_interpolated["Race"]=race[0]
                df_interpolated["Ethinicity"]=ethnicity[0]

                #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

                #Plotting the interpolation

                x=df.index.values
                y=df.Area.values

                xnew=df_interpolated.index.values
                ynew=df_interpolated.Area.values
                plt.plot(x, y, 'o', xnew, ynew, '-')
                plt.plot(x,y,'o')
                plt.show()

                dfB_1=df_interpolated

                first=dfA["Area"].values
                second=dfB_1["Area"].values

                display("Test values from the first interpolation (now considered as actual)",first[start:end])
                display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

                first_test_values=first[start:end]
                second_test_values=second[start:end]

                mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
                rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
                corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
                r_square= corr**2

                print("MAPE:",round(mape,4))
                print("RMSE",round(rmse,4))
                print("R-squarred",round(r_square,4))
                print("AIC",round(AIC_final,4))

                all_mape.append(mape)
                all_rmse.append(rmse)
                all_rsquarred.append(r_square)
                all_aic.append(AIC_final)
                all_wound.append(number)

In [99]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric_new3=pd.DataFrame(lists).transpose()
df_metric_new3.columns=["RMSE","MAPE","AIC","Wound Number","R","Method"]
df_metric_new3

count=0
for i in range(df_metric_new3.shape[0]):
    if df_metric_new3["RMSE"][i]==0 and str(df_metric_new3["MAPE"][i])=='nan':
        df_metric_new3["MAPE"][i]=0
        count=count+1
display(df_metric_new3.describe())
print(count)

count=0
for i in range(df_metric_new3.shape[0]):
    if df_metric_new3["RMSE"][i]==0:
        if df_metric_new3["MAPE"][i]==0:
            if str(df_metric_new3["R"][i])=="nan":
                df_metric_new3["R"][i]=1
                count=count+1
#display(df_metric.describe())
print(count)


# #df_metric_1=df_metric.dropna()
df_metric_new3=df_metric_new3.reset_index()
df_metric_new3=df_metric_new3.drop(["index"],axis=1)
df_metric_new3
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric_new3[(df_metric_new3['RMSE'] >= 5) | (df_metric_new3['MAPE'] >= 5)]
high_mape_df

list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

df_metric_4 = df_metric_new3.drop(df_metric_new3.index[list_index])
df_metric_4.describe()

,RMSE,MAPE,AIC,Wound Number,R,Method
count,434.0,434.0,434.000000,434,409.0,434
unique,422.0,363.0,434.000000,434,380.0,4
top,0.0,inf,5000.702469,42,1.0,Linear
freq,13.0,60.0,1.000000,1,26.0,342


5
9


,RMSE,MAPE,AIC,Wound Number,R,Method
count,118.0,118.0,118.000000,118,111.0,118
unique,106.0,106.0,118.000000,118,83.0,4
top,0.0,0.0,660.135528,46,1.0,Linear
freq,13.0,13.0,1.000000,1,28.0,94


## 5th run of interpolation on 316 wounds

In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
    if (i+1) in high_mapes:
        wound_number=list_wounds[i]
        df_indivi_patient=df_for.query("Wound==@wound_number")
        print(color.BOLD+"\n","Wound",str(i+1)+color.END)
        number=i+1

        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()


        df_patient_1_wound_part_1=df_indivi_patient

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_1)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)
        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

            if len(AIC_list_part_1)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')
            else:
                if min(AIC_list_part_1)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    interpolation_method.append('AKIMA')
                elif min(AIC_list_part_1)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    interpolation_method.append('RBF')

                elif min(AIC_list_part_1)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    interpolation_method.append('Krogh')
                elif min(AIC_list_part_1)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    interpolation_method.append('Linear')



            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            #plt.plot(x,y,'o')
            plt.show()


            dfA=df_interpolated

            start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
            print("Start",start)
            end=start-1+math.ceil(len(dfA.index.values)*0.2)
            print("End",end)
            dfA_1=dfA.drop(dfA.index[start:end])

            data_wound=dfA_1
            df_indivi_patient=data_wound
            gender=df_indivi_patient["Gender"].unique()
            race=df_indivi_patient["Race"].unique()
            ethnicity=df_indivi_patient["Ethinicity"].unique()

            df_patient_1_wound_part_2=df_indivi_patient

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound_part_2)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)

            if df.shape[0]>1:
                print("Number of visit days before interpolation is:",df.shape[0])
                df=df.sort_values(by=['Date'])

                dff=dff.sort_values(by=['Date'])

                s=interpolating_dates(dff,df)

                df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
                df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
                df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
                df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

                print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

                AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
                AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

                if len(AIC_list_part_2)==0:
                    print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                    df_interpolated=df_interpolated_linear
                else:
                    if min(AIC_list_part_2)==AIC_akima:
                        print("For this wound, choosing AKIMA as the best interpolator", "\n")
                        df_interpolated=df_interpolated_akima
                        AIC_final=AIC_akima
                    elif min(AIC_list_part_2)==AIC_rbf:
                        print("For this wound, choosing RBF as the best interpolator", "\n")
                        df_interpolated=df_interpolated_rbf
                        AIC_final=AIC_rbf
                    elif min(AIC_list_part_2)==AIC_krogh:
                        print("For this wound, choosing Krogh as the best interpolator", "\n")
                        df_interpolated=df_interpolated_krogh
                        AIC_final=AIC_krogh
                    elif min(AIC_list_part_2)==AIC_linear:
                        print("For this wound, choosing Linear as the best interpolator", "\n")
                        df_interpolated=df_interpolated_linear
                        AIC_final=AIC_linear

                df=s.to_frame().reset_index()
                df_interpolated["Gender"]=gender[0]
                df_interpolated["Race"]=race[0]
                df_interpolated["Ethinicity"]=ethnicity[0]

                #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

                #Plotting the interpolation

                x=df.index.values
                y=df.Area.values

                xnew=df_interpolated.index.values
                ynew=df_interpolated.Area.values
                plt.plot(x, y, 'o', xnew, ynew, '-')
                plt.plot(x,y,'o')
                plt.show()

                dfB_1=df_interpolated

                first=dfA["Area"].values
                second=dfB_1["Area"].values

                display("Test values from the first interpolation (now considered as actual)",first[start:end])
                display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

                first_test_values=first[start:end]
                second_test_values=second[start:end]

                mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
                rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
                corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
                r_square= corr**2

                print("MAPE:",round(mape,4))
                print("RMSE",round(rmse,4))
                print("R-squarred",round(r_square,4))
                print("AIC",round(AIC_final,4))

                all_mape.append(mape)
                all_rmse.append(rmse)
                all_rsquarred.append(r_square)
                all_aic.append(AIC_final)
                all_wound.append(number)

In [101]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric_new4=pd.DataFrame(lists).transpose()
df_metric_new4.columns=["RMSE","MAPE","AIC","Wound Number","R","Method"]
df_metric_new4

count=0
for i in range(df_metric_new4.shape[0]):
    if df_metric_new4["RMSE"][i]==0 and str(df_metric_new4["MAPE"][i])=='nan':
        df_metric_new4["MAPE"][i]=0
        count=count+1
display(df_metric_new4.describe())
print(count)

count=0
for i in range(df_metric_new4.shape[0]):
    if df_metric_new4["RMSE"][i]==0:
        if df_metric_new4["MAPE"][i]==0:
            if str(df_metric_new4["R"][i])=="nan":
                df_metric_new4["R"][i]=1
                count=count+1
#display(df_metric.describe())
print(count)


# #df_metric_1=df_metric.dropna()
df_metric_new4=df_metric_new4.reset_index()
df_metric_new4=df_metric_new4.drop(["index"],axis=1)
df_metric_new4
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric_new4[(df_metric_new4['RMSE'] >= 5) | (df_metric_new4['MAPE'] >= 5)]
high_mape_df

list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

df_metric_5 = df_metric_new4.drop(df_metric_new4.index[list_index])
df_metric_5.describe()

,RMSE,MAPE,AIC,Wound Number,R,Method
count,316.0,316.0,316.000000,316,304.0,316
unique,314.0,273.0,316.000000,316,289.0,4
top,0.0,inf,5035.214172,42,1.0,Linear
freq,3.0,42.0,1.000000,1,12.0,248


2
3


,RMSE,MAPE,AIC,Wound Number,R,Method
count,50.0,50.0,50.000000,50,49.0,50
unique,48.0,48.0,50.000000,50,42.0,4
top,0.0,0.0,237.158357,460,1.0,Linear
freq,3.0,3.0,1.000000,1,7.0,35


## 6th run of interpolation on 266 wounds

In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
    if (i+1) in high_mapes:
        wound_number=list_wounds[i]
        df_indivi_patient=df_for.query("Wound==@wound_number")
        print(color.BOLD+"\n","Wound",str(i+1)+color.END)
        number=i+1

        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()


        df_patient_1_wound_part_1=df_indivi_patient

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_1)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)
        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

            if len(AIC_list_part_1)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')
            else:
                if min(AIC_list_part_1)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    interpolation_method.append('AKIMA')
                elif min(AIC_list_part_1)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    interpolation_method.append('RBF')

                elif min(AIC_list_part_1)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    interpolation_method.append('Krogh')
                elif min(AIC_list_part_1)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    interpolation_method.append('Linear')



            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            #plt.plot(x,y,'o')
            plt.show()


            dfA=df_interpolated

            start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
            print("Start",start)
            end=start-1+math.ceil(len(dfA.index.values)*0.2)
            print("End",end)
            dfA_1=dfA.drop(dfA.index[start:end])

            data_wound=dfA_1
            df_indivi_patient=data_wound
            gender=df_indivi_patient["Gender"].unique()
            race=df_indivi_patient["Race"].unique()
            ethnicity=df_indivi_patient["Ethinicity"].unique()

            df_patient_1_wound_part_2=df_indivi_patient

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound_part_2)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)

            if df.shape[0]>1:
                print("Number of visit days before interpolation is:",df.shape[0])
                df=df.sort_values(by=['Date'])

                dff=dff.sort_values(by=['Date'])

                s=interpolating_dates(dff,df)

                df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
                df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
                df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
                df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

                print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

                AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
                AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

                if len(AIC_list_part_2)==0:
                    print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                    df_interpolated=df_interpolated_linear
                else:
                    if min(AIC_list_part_2)==AIC_akima:
                        print("For this wound, choosing AKIMA as the best interpolator", "\n")
                        df_interpolated=df_interpolated_akima
                        AIC_final=AIC_akima
                    elif min(AIC_list_part_2)==AIC_rbf:
                        print("For this wound, choosing RBF as the best interpolator", "\n")
                        df_interpolated=df_interpolated_rbf
                        AIC_final=AIC_rbf
                    elif min(AIC_list_part_2)==AIC_krogh:
                        print("For this wound, choosing Krogh as the best interpolator", "\n")
                        df_interpolated=df_interpolated_krogh
                        AIC_final=AIC_krogh
                    elif min(AIC_list_part_2)==AIC_linear:
                        print("For this wound, choosing Linear as the best interpolator", "\n")
                        df_interpolated=df_interpolated_linear
                        AIC_final=AIC_linear

                df=s.to_frame().reset_index()
                df_interpolated["Gender"]=gender[0]
                df_interpolated["Race"]=race[0]
                df_interpolated["Ethinicity"]=ethnicity[0]

                #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

                #Plotting the interpolation

                x=df.index.values
                y=df.Area.values

                xnew=df_interpolated.index.values
                ynew=df_interpolated.Area.values
                plt.plot(x, y, 'o', xnew, ynew, '-')
                plt.plot(x,y,'o')
                plt.show()

                dfB_1=df_interpolated

                first=dfA["Area"].values
                second=dfB_1["Area"].values

                display("Test values from the first interpolation (now considered as actual)",first[start:end])
                display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

                first_test_values=first[start:end]
                second_test_values=second[start:end]

                mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
                rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
                corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
                r_square= corr**2

                print("MAPE:",round(mape,4))
                print("RMSE",round(rmse,4))
                print("R-squarred",round(r_square,4))
                print("AIC",round(AIC_final,4))

                all_mape.append(mape)
                all_rmse.append(rmse)
                all_rsquarred.append(r_square)
                all_aic.append(AIC_final)
                all_wound.append(number)

In [103]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric_new5=pd.DataFrame(lists).transpose()
df_metric_new5.columns=["RMSE","MAPE","AIC","Wound Number","R","Method"]
df_metric_new5

count=0
for i in range(df_metric_new5.shape[0]):
    if df_metric_new5["RMSE"][i]==0 and str(df_metric_new5["MAPE"][i])=='nan':
        df_metric_new5["MAPE"][i]=0
        count=count+1
display(df_metric_new5.describe())
print(count)

count=0
for i in range(df_metric_new5.shape[0]):
    if df_metric_new5["RMSE"][i]==0:
        if df_metric_new5["MAPE"][i]==0:
            if str(df_metric_new5["R"][i])=="nan":
                df_metric_new5["R"][i]=1
                count=count+1
#display(df_metric.describe())
print(count)


# #df_metric_1=df_metric.dropna()
df_metric_new5=df_metric_new5.reset_index()
df_metric_new5=df_metric_new5.drop(["index"],axis=1)
df_metric_new5
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric_new5[(df_metric_new5['RMSE'] >= 5) | (df_metric_new5['MAPE'] >= 5)]
high_mape_df

list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

df_metric_6 = df_metric_new5.drop(df_metric_new5.index[list_index])
df_metric_6.describe()

,RMSE,MAPE,AIC,Wound Number,R,Method
count,266.0,265.0,266.000000,266,255.0,266
unique,265.0,236.0,266.000000,266,249.0,4
top,0.0,inf,4998.602718,42,1.0,Linear
freq,2.0,29.0,1.000000,1,6.0,213


0
1


,RMSE,MAPE,AIC,Wound Number,R,Method
count,39.0,38.0,39.000000,39,36.0,39
unique,38.0,37.0,39.000000,39,31.0,4
top,0.0,0.0,7495.365659,121,1.0,Linear
freq,2.0,2.0,1.000000,1,6.0,29


## So even after re-running the interpolation, we are getting high MAPE and high RMSE for some wounds (n=227), what we can do is replace the original 0 value for the observations with 1e-3 and the run the interpolation

In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
    if (i+1) in high_mapes:
        wound_number=list_wounds[i]
        df_indivi_patient=df_for.query("Wound==@wound_number")
        display(df_indivi_patient)
        
        #Replacing 0 area value
        
        for j in range(df_indivi_patient.index[0],df_indivi_patient.index[0]+len(df_indivi_patient.index)):
            if df_indivi_patient["Area"][j]==0:
                df_indivi_patient["Area"][j]=1e-3
        display(df_indivi_patient)
        
        
        print(color.BOLD+"\n","Wound",str(i+1)+color.END)
        number=i+1

        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()


        df_patient_1_wound_part_1=df_indivi_patient

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_1)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)
        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

            if len(AIC_list_part_1)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')
            else:
                if min(AIC_list_part_1)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    interpolation_method.append('AKIMA')
                elif min(AIC_list_part_1)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    interpolation_method.append('RBF')

                elif min(AIC_list_part_1)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    interpolation_method.append('Krogh')
                elif min(AIC_list_part_1)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    interpolation_method.append('Linear')



            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            #plt.plot(x,y,'o')
            plt.show()


            dfA=df_interpolated

            start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
            print("Start",start)
            end=start-1+math.ceil(len(dfA.index.values)*0.2)
            print("End",end)
            dfA_1=dfA.drop(dfA.index[start:end])

            data_wound=dfA_1
            df_indivi_patient=data_wound
            gender=df_indivi_patient["Gender"].unique()
            race=df_indivi_patient["Race"].unique()
            ethnicity=df_indivi_patient["Ethinicity"].unique()

            df_patient_1_wound_part_2=df_indivi_patient

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound_part_2)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)

            if df.shape[0]>1:
                print("Number of visit days before interpolation is:",df.shape[0])
                df=df.sort_values(by=['Date'])

                dff=dff.sort_values(by=['Date'])

                s=interpolating_dates(dff,df)

                df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
                df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
                df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
                df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

                print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

                AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
                AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

                if len(AIC_list_part_2)==0:
                    print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                    df_interpolated=df_interpolated_linear
                else:
                    if min(AIC_list_part_2)==AIC_akima:
                        print("For this wound, choosing AKIMA as the best interpolator", "\n")
                        df_interpolated=df_interpolated_akima
                        AIC_final=AIC_akima
                    elif min(AIC_list_part_2)==AIC_rbf:
                        print("For this wound, choosing RBF as the best interpolator", "\n")
                        df_interpolated=df_interpolated_rbf
                        AIC_final=AIC_rbf
                    elif min(AIC_list_part_2)==AIC_krogh:
                        print("For this wound, choosing Krogh as the best interpolator", "\n")
                        df_interpolated=df_interpolated_krogh
                        AIC_final=AIC_krogh
                    elif min(AIC_list_part_2)==AIC_linear:
                        print("For this wound, choosing Linear as the best interpolator", "\n")
                        df_interpolated=df_interpolated_linear
                        AIC_final=AIC_linear

                df=s.to_frame().reset_index()
                df_interpolated["Gender"]=gender[0]
                df_interpolated["Race"]=race[0]
                df_interpolated["Ethinicity"]=ethnicity[0]

                #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

                #Plotting the interpolation

                x=df.index.values
                y=df.Area.values

                xnew=df_interpolated.index.values
                ynew=df_interpolated.Area.values
                plt.plot(x, y, 'o', xnew, ynew, '-')
                plt.plot(x,y,'o')
                plt.show()

                dfB_1=df_interpolated

                first=dfA["Area"].values
                second=dfB_1["Area"].values

                display("Test values from the first interpolation (now considered as actual)",first[start:end])
                display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

                first_test_values=first[start:end]
                second_test_values=second[start:end]

                mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
                rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
                corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
                r_square= corr**2

                print("MAPE:",round(mape,4))
                print("RMSE",round(rmse,4))
                print("R-squarred",round(r_square,4))
                print("AIC",round(AIC_final,4))

                all_mape.append(mape)
                all_rmse.append(rmse)
                all_rsquarred.append(r_square)
                all_aic.append(AIC_final)
                all_wound.append(number)

In [106]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric_new6=pd.DataFrame(lists).transpose()
df_metric_new6.columns=["RMSE","MAPE","AIC","Wound Number","R","Method"]
df_metric_new6

count=0
for i in range(df_metric_new6.shape[0]):
    if df_metric_new6["RMSE"][i]==0 and str(df_metric_new6["MAPE"][i])=='nan':
        df_metric_new6["MAPE"][i]=0
        count=count+1
display(df_metric_new6.describe())
print(count)

count=0
for i in range(df_metric_new6.shape[0]):
    if df_metric_new6["RMSE"][i]==0:
        if df_metric_new6["MAPE"][i]==0:
            if str(df_metric_new6["R"][i])=="nan":
                df_metric_new6["R"][i]=1
                count=count+1
#display(df_metric.describe())
print(count)


# #df_metric_1=df_metric.dropna()
df_metric_new6=df_metric_new6.reset_index()
df_metric_new6=df_metric_new6.drop(["index"],axis=1)
df_metric_new6
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric_new6[(df_metric_new6['RMSE'] >= 5) | (df_metric_new6['MAPE'] >= 5)]
high_mape_df

list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

df_metric_7 = df_metric_new6.drop(df_metric_new6.index[list_index])
df_metric_7.describe()

,RMSE,MAPE,AIC,Wound Number,R,Method
count,227.000000,227.000000,227.000000,227,220.0,227
unique,227.000000,227.000000,227.000000,227,214.0,4
top,26.774048,0.340143,5014.745216,42,1.0,Linear
freq,1.000000,1.000000,1.000000,1,7.0,184


0
1


,RMSE,MAPE,AIC,Wound Number,R,Method
count,36.00000,36.000000,36.000000,36,33.0,36
unique,36.00000,36.000000,36.000000,36,30.0,3
top,0.55863,0.251743,2721.299898,431,1.0,Linear
freq,1.00000,1.000000,1.000000,1,4.0,27


## Re-run again (for n=191)

In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
    if (i+1) in high_mapes:
        wound_number=list_wounds[i]
        df_indivi_patient=df_for.query("Wound==@wound_number")
        display(df_indivi_patient)
        
        #Replacing 0 area value
        
        for j in range(df_indivi_patient.index[0],df_indivi_patient.index[0]+len(df_indivi_patient.index)):
            if df_indivi_patient["Area"][j]==0:
                df_indivi_patient["Area"][j]=1e-3
        display(df_indivi_patient)
        
        
        print(color.BOLD+"\n","Wound",str(i+1)+color.END)
        number=i+1

        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()


        df_patient_1_wound_part_1=df_indivi_patient

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_1)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)
        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

            if len(AIC_list_part_1)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')
            else:
                if min(AIC_list_part_1)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    interpolation_method.append('AKIMA')
                elif min(AIC_list_part_1)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    interpolation_method.append('RBF')

                elif min(AIC_list_part_1)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    interpolation_method.append('Krogh')
                elif min(AIC_list_part_1)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    interpolation_method.append('Linear')



            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            #plt.plot(x,y,'o')
            plt.show()


            dfA=df_interpolated

            start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
            print("Start",start)
            end=start-1+math.ceil(len(dfA.index.values)*0.2)
            print("End",end)
            dfA_1=dfA.drop(dfA.index[start:end])

            data_wound=dfA_1
            df_indivi_patient=data_wound
            gender=df_indivi_patient["Gender"].unique()
            race=df_indivi_patient["Race"].unique()
            ethnicity=df_indivi_patient["Ethinicity"].unique()

            df_patient_1_wound_part_2=df_indivi_patient

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound_part_2)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)

            if df.shape[0]>1:
                print("Number of visit days before interpolation is:",df.shape[0])
                df=df.sort_values(by=['Date'])

                dff=dff.sort_values(by=['Date'])

                s=interpolating_dates(dff,df)

                df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
                df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
                df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
                df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

                print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

                AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
                AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

                if len(AIC_list_part_2)==0:
                    print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                    df_interpolated=df_interpolated_linear
                else:
                    if min(AIC_list_part_2)==AIC_akima:
                        print("For this wound, choosing AKIMA as the best interpolator", "\n")
                        df_interpolated=df_interpolated_akima
                        AIC_final=AIC_akima
                    elif min(AIC_list_part_2)==AIC_rbf:
                        print("For this wound, choosing RBF as the best interpolator", "\n")
                        df_interpolated=df_interpolated_rbf
                        AIC_final=AIC_rbf
                    elif min(AIC_list_part_2)==AIC_krogh:
                        print("For this wound, choosing Krogh as the best interpolator", "\n")
                        df_interpolated=df_interpolated_krogh
                        AIC_final=AIC_krogh
                    elif min(AIC_list_part_2)==AIC_linear:
                        print("For this wound, choosing Linear as the best interpolator", "\n")
                        df_interpolated=df_interpolated_linear
                        AIC_final=AIC_linear

                df=s.to_frame().reset_index()
                df_interpolated["Gender"]=gender[0]
                df_interpolated["Race"]=race[0]
                df_interpolated["Ethinicity"]=ethnicity[0]

                #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

                #Plotting the interpolation

                x=df.index.values
                y=df.Area.values

                xnew=df_interpolated.index.values
                ynew=df_interpolated.Area.values
                plt.plot(x, y, 'o', xnew, ynew, '-')
                plt.plot(x,y,'o')
                plt.show()

                dfB_1=df_interpolated

                first=dfA["Area"].values
                second=dfB_1["Area"].values

                display("Test values from the first interpolation (now considered as actual)",first[start:end])
                display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

                first_test_values=first[start:end]
                second_test_values=second[start:end]

                mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
                rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
                corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
                r_square= corr**2

                print("MAPE:",round(mape,4))
                print("RMSE",round(rmse,4))
                print("R-squarred",round(r_square,4))
                print("AIC",round(AIC_final,4))

                all_mape.append(mape)
                all_rmse.append(rmse)
                all_rsquarred.append(r_square)
                all_aic.append(AIC_final)
                all_wound.append(number)

In [108]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric_new7=pd.DataFrame(lists).transpose()
df_metric_new7.columns=["RMSE","MAPE","AIC","Wound Number","R","Method"]
df_metric_new7

count=0
for i in range(df_metric_new7.shape[0]):
    if df_metric_new7["RMSE"][i]==0 and str(df_metric_new7["MAPE"][i])=='nan':
        df_metric_new7["MAPE"][i]=0
        count=count+1
display(df_metric_new7.describe())
print(count)

count=0
for i in range(df_metric_new7.shape[0]):
    if df_metric_new7["RMSE"][i]==0:
        if df_metric_new7["MAPE"][i]==0:
            if str(df_metric_new7["R"][i])=="nan":
                df_metric_new7["R"][i]=1
                count=count+1
#display(df_metric.describe())
print(count)


# #df_metric_1=df_metric.dropna()
df_metric_new7=df_metric_new7.reset_index()
df_metric_new7=df_metric_new7.drop(["index"],axis=1)
df_metric_new7
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric_new7[(df_metric_new7['RMSE'] >= 5) | (df_metric_new7['MAPE'] >= 5)]
high_mape_df

list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

df_metric_8 = df_metric_new7.drop(df_metric_new7.index[list_index])
df_metric_8.describe()

,RMSE,MAPE,AIC,Wound Number,R,Method
count,191.000000,191.000000,191.000000,191,184.0,191
unique,191.000000,191.000000,191.000000,191,182.0,4
top,27.165915,0.162083,4960.654487,42,1.0,Linear
freq,1.000000,1.000000,1.000000,1,2.0,157


0
1


,RMSE,MAPE,AIC,Wound Number,R,Method
count,27.000000,27.000000,27.00000,27,27.000000,27
unique,27.000000,27.000000,27.00000,27,27.000000,4
top,1.092544,0.136137,421.87096,1667,0.982929,Linear
freq,1.000000,1.000000,1.00000,1,1.000000,20


## Re-run again (2) (n=164)

In [ ]:
all_mape=[]
all_rmse=[]
all_rsquarred=[]
all_aic=[]
all_wound=[]

interpolation_method=[]

list_wounds=df_for["Wound"].unique().tolist()
#list_wounds=[]
wound=0
df_merged=pd.DataFrame()

for i in range(len(list_wounds)):
#     if i>50:
#         break
    if (i+1) in high_mapes:
        wound_number=list_wounds[i]
        df_indivi_patient=df_for.query("Wound==@wound_number")
        display(df_indivi_patient)
        
        #Replacing 0 area value
        
        for j in range(df_indivi_patient.index[0],df_indivi_patient.index[0]+len(df_indivi_patient.index)):
            if df_indivi_patient["Area"][j]==0:
                df_indivi_patient["Area"][j]=1e-3
        display(df_indivi_patient)
        
        
        print(color.BOLD+"\n","Wound",str(i+1)+color.END)
        number=i+1

        gender=df_indivi_patient["Gender"].unique()
        race=df_indivi_patient["Race"].unique()
        ethnicity=df_indivi_patient["Ethinicity"].unique()


        df_patient_1_wound_part_1=df_indivi_patient

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.sort_values(by=['Date'])
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)

        df_patient_1_wound_part_1=df_patient_1_wound_part_1.dropna()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.reset_index()
        df_patient_1_wound_part_1=df_patient_1_wound_part_1.drop(["index"],axis=1)




        #Do the interpolation for all dates
        area, date= get_date_and_area(df_patient_1_wound_part_1)    
        #print(area,date)
        dff,df=indexing_dataframe(date,area)
        if df.shape[0]>1:
            print("Number of visit days before interpolation is:",df.shape[0])
            df=df.sort_values(by=['Date'])

            dff=dff.sort_values(by=['Date'])

            s=interpolating_dates(dff,df)

            df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
            df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
            df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
            df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

            print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

            AIC_list_part_1=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
            AIC_list_part_1=[x for x in AIC_list_part_1 if str(x) != 'nan']

            if len(AIC_list_part_1)==0:
                print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                df_interpolated=df_interpolated_linear
                interpolation_method.append('Linear')
            else:
                if min(AIC_list_part_1)==AIC_akima:
                    print("For this wound, choosing AKIMA as the best interpolator", "\n")
                    df_interpolated=df_interpolated_akima
                    interpolation_method.append('AKIMA')
                elif min(AIC_list_part_1)==AIC_rbf:
                    print("For this wound, choosing RBF as the best interpolator", "\n")
                    df_interpolated=df_interpolated_rbf
                    interpolation_method.append('RBF')

                elif min(AIC_list_part_1)==AIC_krogh:
                    print("For this wound, choosing Krogh as the best interpolator", "\n")
                    df_interpolated=df_interpolated_krogh
                    interpolation_method.append('Krogh')
                elif min(AIC_list_part_1)==AIC_linear:
                    print("For this wound, choosing Linear as the best interpolator", "\n")
                    df_interpolated=df_interpolated_linear
                    interpolation_method.append('Linear')



            df=s.to_frame().reset_index()
            df_interpolated["Gender"]=gender[0]
            df_interpolated["Race"]=race[0]
            df_interpolated["Ethinicity"]=ethnicity[0]

            df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

            #Plotting the interpolation

            x=df.index.values
            y=df.Area.values

            xnew=df_interpolated.index.values
            ynew=df_interpolated.Area.values
            plt.plot(x, y, 'o', xnew, ynew, '-')
            #plt.plot(x,y,'o')
            plt.show()


            dfA=df_interpolated

            start=random.randint(0,(len(dfA.index.values)-int(len(dfA.index.values)*0.2))-1)        
            print("Start",start)
            end=start-1+math.ceil(len(dfA.index.values)*0.2)
            print("End",end)
            dfA_1=dfA.drop(dfA.index[start:end])

            data_wound=dfA_1
            df_indivi_patient=data_wound
            gender=df_indivi_patient["Gender"].unique()
            race=df_indivi_patient["Race"].unique()
            ethnicity=df_indivi_patient["Ethinicity"].unique()

            df_patient_1_wound_part_2=df_indivi_patient

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.sort_values(by=['Date'])
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)

            df_patient_1_wound_part_2=df_patient_1_wound_part_2.dropna()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.reset_index()
            df_patient_1_wound_part_2=df_patient_1_wound_part_2.drop(["index"],axis=1)




            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound_part_2)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)

            if df.shape[0]>1:
                print("Number of visit days before interpolation is:",df.shape[0])
                df=df.sort_values(by=['Date'])

                dff=dff.sort_values(by=['Date'])

                s=interpolating_dates(dff,df)

                df_interpolated_akima, AIC_akima=interpolating_areas_akima(s)
                df_interpolated_rbf, AIC_rbf=interpolating_areas_rbf(s)
                df_interpolated_krogh, AIC_krogh=interpolating_areas_krogh(s)
                df_interpolated_linear, AIC_linear=interpolating_areas_linear(s)

                print("\n","AIC_Akima:",AIC_akima, "\n", "AIC_RBF:", AIC_rbf, "\n", "AIC_Krogh:", AIC_krogh,"\n","AIC_linear:", AIC_linear)#, "AIC_polyfit", AIC_poly)

                AIC_list_part_2=[AIC_akima, AIC_rbf, AIC_krogh, AIC_linear]#, AIC_poly]
                AIC_list_part_2=[x for x in AIC_list_part_2 if str(x) != 'nan']

                if len(AIC_list_part_2)==0:
                    print("AIC could not be calculated, thus choosing Linear as the interpolator","\n")
                    df_interpolated=df_interpolated_linear
                else:
                    if min(AIC_list_part_2)==AIC_akima:
                        print("For this wound, choosing AKIMA as the best interpolator", "\n")
                        df_interpolated=df_interpolated_akima
                        AIC_final=AIC_akima
                    elif min(AIC_list_part_2)==AIC_rbf:
                        print("For this wound, choosing RBF as the best interpolator", "\n")
                        df_interpolated=df_interpolated_rbf
                        AIC_final=AIC_rbf
                    elif min(AIC_list_part_2)==AIC_krogh:
                        print("For this wound, choosing Krogh as the best interpolator", "\n")
                        df_interpolated=df_interpolated_krogh
                        AIC_final=AIC_krogh
                    elif min(AIC_list_part_2)==AIC_linear:
                        print("For this wound, choosing Linear as the best interpolator", "\n")
                        df_interpolated=df_interpolated_linear
                        AIC_final=AIC_linear

                df=s.to_frame().reset_index()
                df_interpolated["Gender"]=gender[0]
                df_interpolated["Race"]=race[0]
                df_interpolated["Ethinicity"]=ethnicity[0]

                #df_merged = pd.concat([df_merged, df_interpolated], ignore_index=True, sort=False)

                #Plotting the interpolation

                x=df.index.values
                y=df.Area.values

                xnew=df_interpolated.index.values
                ynew=df_interpolated.Area.values
                plt.plot(x, y, 'o', xnew, ynew, '-')
                plt.plot(x,y,'o')
                plt.show()

                dfB_1=df_interpolated

                first=dfA["Area"].values
                second=dfB_1["Area"].values

                display("Test values from the first interpolation (now considered as actual)",first[start:end])
                display("Test values from the second interpolation (now considered as forecasted value)",second[start:end])

                first_test_values=first[start:end]
                second_test_values=second[start:end]

                mape = np.mean(np.abs(second_test_values - first_test_values)/np.abs(first_test_values))  # MAPE
                rmse = np.mean((second_test_values - first_test_values)**2)**.5  # RMSE
                corr = np.corrcoef(second_test_values, first_test_values)[0,1]   # corr
                r_square= corr**2

                print("MAPE:",round(mape,4))
                print("RMSE",round(rmse,4))
                print("R-squarred",round(r_square,4))
                print("AIC",round(AIC_final,4))

                all_mape.append(mape)
                all_rmse.append(rmse)
                all_rsquarred.append(r_square)
                all_aic.append(AIC_final)
                all_wound.append(number)

In [110]:
lists=[all_rmse,all_mape,all_aic,all_wound,all_rsquarred,interpolation_method]
df_metric_new8=pd.DataFrame(lists).transpose()
df_metric_new8.columns=["RMSE","MAPE","AIC","Wound Number","R","Method"]
df_metric_new8

count=0
for i in range(df_metric_new8.shape[0]):
    if df_metric_new8["RMSE"][i]==0 and str(df_metric_new8["MAPE"][i])=='nan':
        df_metric_new8["MAPE"][i]=0
        count=count+1
display(df_metric_new8.describe())
print(count)

count=0
for i in range(df_metric_new8.shape[0]):
    if df_metric_new8["RMSE"][i]==0:
        if df_metric_new8["MAPE"][i]==0:
            if str(df_metric_new8["R"][i])=="nan":
                df_metric_new8["R"][i]=1
                count=count+1
#display(df_metric.describe())
print(count)


# #df_metric_1=df_metric.dropna()
df_metric_new8=df_metric_new8.reset_index()
df_metric_new8=df_metric_new8.drop(["index"],axis=1)
df_metric_new8
# high_mape_df=df_metric.sort_values('MAPE').tail(717)
# high_mape_df
high_mape_df=df_metric_new8[(df_metric_new8['RMSE'] >= 5) | (df_metric_new8['MAPE'] >= 5)]
high_mape_df

list_index=high_mape_df.index.values.tolist()
high_mapes=list(map(int, high_mape_df["Wound Number"].values.tolist()))
high_mapes.sort()

df_metric_9 = df_metric_new8.drop(df_metric_new8.index[list_index])
df_metric_9.describe()

,RMSE,MAPE,AIC,Wound Number,R,Method
count,164.00000,164.000000,164.000000,164,159.000000,164
unique,164.00000,164.000000,164.000000,164,159.000000,4
top,48.56192,0.647122,5016.580006,42,0.181907,Linear
freq,1.00000,1.000000,1.000000,1,1.000000,137


0
1


,RMSE,MAPE,AIC,Wound Number,R,Method
count,10.000000,10.000000,10.000000,10,10.0,10
unique,10.000000,10.000000,10.000000,10,9.0,1
top,2.446077,0.014684,3441.979476,3674,1.0,Linear
freq,1.000000,1.000000,1.000000,1,2.0,10


In [114]:
data = pd.concat([df_metric_1, df_metric_2, df_metric_3, df_metric_4, df_metric_5, df_metric_6, df_metric_7, df_metric_8, df_metric_9], axis=0)
data=data.sort_values('Wound Number')
data=data.reset_index()
data=data.drop(['index'],axis=1)
data

,RMSE,MAPE,AIC,Wound Number,R,Method
0,1.156098,0.638773,576.691058,1,0.639338,Linear
1,0.0,0.0,278.083492,2,1.0,Linear
2,0.006555,0.007433,-43.18967,3,1.0,RBF
3,0.024748,0.017861,53.521066,5,0.999942,Linear
4,0.026651,0.068398,11.67976,6,0.997253,Krogh
...,...,...,...,...,...,...
14412,0.0,0.0,36.477264,17877,1.0,Linear
14413,0.004448,0.011955,-38.374363,17879,NaN,RBF
14414,0.0,0.0,57.542496,17880,1.0,Linear
14415,1.504483,0.010111,69.475556,17881,NaN,RBF


In [134]:
#data1=data.drop(["Method"],axis=1)
display(data.mean())
display(data.std())
display(data["Method"].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')

RMSE               0.450126
MAPE               0.198537
AIC              294.074023
Wound Number    9272.194493
R                  0.835359
dtype: float64

RMSE               0.916497
MAPE               0.507617
AIC              946.653950
Wound Number    5107.463751
R                  0.268749
dtype: float64

Linear    65.35%
Krogh     15.47%
AKIMA     13.41%
RBF        5.77%
Name: Method, dtype: object

In [124]:
data.to_csv("good_metric.csv")

In [126]:
high_mape_df.to_csv("bad_metric.csv")

In [128]:
high_mape_df

,RMSE,MAPE,AIC,Wound Number,R,Method
0,48.56192,0.647122,5016.580006,42,0.181907,Linear
1,269.87673,0.655661,9026.920381,79,0.222374,Linear
2,56.747896,1.590182,5464.483203,80,0.797941,Linear
3,13.300089,0.169916,1882.479418,81,0.682203,Linear
4,52.479002,0.254897,4284.235599,125,0.000387,Linear
...,...,...,...,...,...,...
159,53.087742,0.396279,3067.797076,17038,0.84773,Linear
160,145.022662,5.697676,2275.812132,17285,0.298455,Linear
161,75.617964,0.596238,2083.35726,17286,0.032781,Linear
162,13.089941,0.109167,2116.365985,17287,0.074624,Linear


In [133]:
display(high_mape_df.mean())
display(high_mape_df.std())
display(high_mape_df["Method"].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')

RMSE               44.875940
MAPE              300.797506
AIC              3576.049142
Wound Number    10074.798701
R                   0.457428
dtype: float64

RMSE              60.494059
MAPE            1867.021370
AIC             3722.802888
Wound Number    5244.652377
R                  0.321136
dtype: float64

Linear    82.47%
RBF       10.39%
AKIMA      5.84%
Krogh       1.3%
Name: Method, dtype: object

In [2]:
good=pd.read_csv("good_metric.csv")
bad=pd.read_csv("bad_metric.csv")

In [3]:
data1 = pd.concat([good, bad], axis=0)


In [5]:
display(data1.mean())
display(data1.std())
display(data1["Method"].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')

Unnamed: 0      7132.696177
RMSE               0.919660
MAPE               3.376203
AIC              328.761016
Wound Number    9280.677167
R                  0.831162
dtype: float64

Unnamed: 0      4203.554234
RMSE               7.739529
MAPE             193.796607
AIC             1069.983439
Wound Number    5109.407577
R                  0.272268
dtype: float64

Linear    65.53%
Krogh     15.32%
AKIMA     13.33%
RBF        5.82%
Name: Method, dtype: object